In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
%%bash
cd /kaggle/working
pip install -q -U -t /kaggle/working/submission/lib immutabledict sentencepiece
git clone https://github.com/google/gemma_pytorch.git > /dev/null
mkdir /kaggle/working/submission/lib/gemma/
mv /kaggle/working/gemma_pytorch/gemma/* /kaggle/working/submission/lib/gemma/

In [ ]:
import json
import os
import pandas as pd
import random
import string
import torch

from os import path
from pathlib import Path
from random import choice
from string import Template
from transformers import T5Tokenizer, T5ForConditionalGeneration


KEYWORDS_JSON = """
[
  {
    "category": "country",
    "words": [
      {
        "keyword": "afghanistan",
        "alts": []
      },
      {
        "keyword": "albania",
        "alts": []
      },
      {
        "keyword": "algeria",
        "alts": []
      },
      {
        "keyword": "andorra",
        "alts": []
      },
      {
        "keyword": "angola",
        "alts": []
      },
      {
        "keyword": "antigua and barbuda",
        "alts": []
      },
      {
        "keyword": "argentina",
        "alts": []
      },
      {
        "keyword": "armenia",
        "alts": []
      },
      {
        "keyword": "australia",
        "alts": []
      },
      {
        "keyword": "austria",
        "alts": []
      },
      {
        "keyword": "azerbaijan",
        "alts": []
      },
      {
        "keyword": "bahrain",
        "alts": []
      },
      {
        "keyword": "bangladesh",
        "alts": []
      },
      {
        "keyword": "barbados",
        "alts": []
      },
      {
        "keyword": "belarus",
        "alts": []
      },
      {
        "keyword": "belgium",
        "alts": []
      },
      {
        "keyword": "belize",
        "alts": []
      },
      {
        "keyword": "benin",
        "alts": []
      },
      {
        "keyword": "bhutan",
        "alts": []
      },
      {
        "keyword": "bolivia",
        "alts": []
      },
      {
        "keyword": "bosnia and herzegovina",
        "alts": []
      },
      {
        "keyword": "botswana",
        "alts": []
      },
      {
        "keyword": "brazil",
        "alts": []
      },
      {
        "keyword": "brunei",
        "alts": []
      },
      {
        "keyword": "bulgaria",
        "alts": []
      },
      {
        "keyword": "burkina faso",
        "alts": []
      },
      {
        "keyword": "burundi",
        "alts": []
      },
      {
        "keyword": "cambodia",
        "alts": []
      },
      {
        "keyword": "cameroon",
        "alts": []
      },
      {
        "keyword": "canada",
        "alts": []
      },
      {
        "keyword": "cape verde",
        "alts": ["cabo verde"]
      },
      {
        "keyword": "central african republic",
        "alts": []
      },
      {
        "keyword": "chad",
        "alts": []
      },
      {
        "keyword": "chile",
        "alts": []
      },
      {
        "keyword": "china",
        "alts": []
      },
      {
        "keyword": "colombia",
        "alts": []
      },
      {
        "keyword": "comoros",
        "alts": []
      },
      {
        "keyword": "congo",
        "alts": ["republic of the congo", "congo brazzaville", "congo republic"]
      },
      {
        "keyword": "costa rica",
        "alts": []
      },
      {
        "keyword": "croatia",
        "alts": []
      },
      {
        "keyword": "cuba",
        "alts": []
      },
      {
        "keyword": "cyprus",
        "alts": []
      },
      {
        "keyword": "czech republic",
        "alts": []
      },
      {
        "keyword": "democratic republic of the congo",
        "alts": ["drc", "congo kinshasa"]
      },
      {
        "keyword": "denmark",
        "alts": []
      },
      {
        "keyword": "djibouti",
        "alts": []
      },
      {
        "keyword": "dominica",
        "alts": []
      },
      {
        "keyword": "dominican republic",
        "alts": []
      },
      {
        "keyword": "ecuador",
        "alts": []
      },
      {
        "keyword": "egypt",
        "alts": []
      },
      {
        "keyword": "el salvador",
        "alts": []
      },
      {
        "keyword": "england",
        "alts": []
      },
      {
        "keyword": "equatorial guinea",
        "alts": []
      },
      {
        "keyword": "eritrea",
        "alts": []
      },
      {
        "keyword": "estonia",
        "alts": []
      },
      {
        "keyword": "ethiopia",
        "alts": []
      },
      {
        "keyword": "federated states of micronesia",
        "alts": ["micronesia"]
      },
      {
        "keyword": "finland",
        "alts": []
      },
      {
        "keyword": "france",
        "alts": []
      },
      {
        "keyword": "gabon",
        "alts": []
      },
      {
        "keyword": "gambia",
        "alts": []
      },
      {
        "keyword": "georgia",
        "alts": []
      },
      {
        "keyword": "germany",
        "alts": []
      },
      {
        "keyword": "ghana",
        "alts": []
      },
      {
        "keyword": "greece",
        "alts": []
      },
      {
        "keyword": "grenada",
        "alts": []
      },
      {
        "keyword": "guatemala",
        "alts": []
      },
      {
        "keyword": "guinea",
        "alts": []
      },
      {
        "keyword": "guinea bissau",
        "alts": []
      },
      {
        "keyword": "guyana",
        "alts": []
      },
      {
        "keyword": "haiti",
        "alts": []
      },
      {
        "keyword": "honduras",
        "alts": []
      },
      {
        "keyword": "hungary",
        "alts": []
      },
      {
        "keyword": "iceland",
        "alts": []
      },
      {
        "keyword": "india",
        "alts": []
      },
      {
        "keyword": "indonesia",
        "alts": []
      },
      {
        "keyword": "iran",
        "alts": []
      },
      {
        "keyword": "iraq",
        "alts": []
      },
      {
        "keyword": "ireland",
        "alts": []
      },
      {
        "keyword": "israel",
        "alts": []
      },
      {
        "keyword": "italy",
        "alts": []
      },
      {
        "keyword": "jamaica",
        "alts": []
      },
      {
        "keyword": "japan",
        "alts": []
      },
      {
        "keyword": "jordan",
        "alts": []
      },
      {
        "keyword": "kazakhstan",
        "alts": []
      },
      {
        "keyword": "kenya",
        "alts": []
      },
      {
        "keyword": "kiribati",
        "alts": []
      },
      {
        "keyword": "kosovo",
        "alts": []
      },
      {
        "keyword": "kuwait",
        "alts": []
      },
      {
        "keyword": "kyrgyzstan",
        "alts": []
      },
      {
        "keyword": "laos",
        "alts": []
      },
      {
        "keyword": "latvia",
        "alts": []
      },
      {
        "keyword": "lebanon",
        "alts": []
      },
      {
        "keyword": "lesotho",
        "alts": []
      },
      {
        "keyword": "liberia",
        "alts": []
      },
      {
        "keyword": "libya",
        "alts": []
      },
      {
        "keyword": "liechtenstein",
        "alts": []
      },
      {
        "keyword": "lithuania",
        "alts": []
      },
      {
        "keyword": "luxembourg",
        "alts": []
      },
      {
        "keyword": "madagascar",
        "alts": []
      },
      {
        "keyword": "malawi",
        "alts": []
      },
      {
        "keyword": "malaysia",
        "alts": []
      },
      {
        "keyword": "maldives",
        "alts": []
      },
      {
        "keyword": "mali",
        "alts": []
      },
      {
        "keyword": "malta",
        "alts": []
      },
      {
        "keyword": "marshall islands",
        "alts": []
      },
      {
        "keyword": "mauritania",
        "alts": []
      },
      {
        "keyword": "mauritius",
        "alts": []
      },
      {
        "keyword": "mexico",
        "alts": []
      },
      {
        "keyword": "moldova",
        "alts": []
      },
      {
        "keyword": "monaco",
        "alts": []
      },
      {
        "keyword": "mongolia",
        "alts": []
      },
      {
        "keyword": "montenegro",
        "alts": []
      },
      {
        "keyword": "morocco",
        "alts": []
      },
      {
        "keyword": "mozambique",
        "alts": []
      },
      {
        "keyword": "myanmar",
        "alts": ["burma"]
      },
      {
        "keyword": "namibia",
        "alts": []
      },
      {
        "keyword": "nauru",
        "alts": []
      },
      {
        "keyword": "nepal",
        "alts": []
      },
      {
        "keyword": "netherlands",
        "alts": ["holland"]
      },
      {
        "keyword": "new zealand",
        "alts": []
      },
      {
        "keyword": "nicaragua",
        "alts": []
      },
      {
        "keyword": "niger",
        "alts": []
      },
      {
        "keyword": "nigeria",
        "alts": []
      },
      {
        "keyword": "north korea",
        "alts": ["dprk", "democratic peoples republic of korea"]
      },
      {
        "keyword": "norway",
        "alts": []
      },
      {
        "keyword": "oman",
        "alts": []
      },
      {
        "keyword": "pakistan",
        "alts": []
      },
      {
        "keyword": "palau",
        "alts": []
      },
      {
        "keyword": "palestine",
        "alts": []
      },
      {
        "keyword": "panama",
        "alts": []
      },
      {
        "keyword": "papua new guinea",
        "alts": []
      },
      {
        "keyword": "paraguay",
        "alts": []
      },
      {
        "keyword": "peru",
        "alts": []
      },
      {
        "keyword": "philippines",
        "alts": []
      },
      {
        "keyword": "poland",
        "alts": []
      },
      {
        "keyword": "portugal",
        "alts": []
      },
      {
        "keyword": "qatar",
        "alts": []
      },
      {
        "keyword": "romania",
        "alts": []
      },
      {
        "keyword": "russia",
        "alts": []
      },
      {
        "keyword": "rwanda",
        "alts": []
      },
      {
        "keyword": "saint kitts and nevis",
        "alts": []
      },
      {
        "keyword": "saint lucia",
        "alts": []
      },
      {
        "keyword": "saint vincent and the grenadines",
        "alts": []
      },
      {
        "keyword": "samoa",
        "alts": []
      },
      {
        "keyword": "san marino",
        "alts": []
      },
      {
        "keyword": "sao tome and principe",
        "alts": []
      },
      {
        "keyword": "saudi arabia",
        "alts": []
      },
      {
        "keyword": "senegal",
        "alts": []
      },
      {
        "keyword": "serbia",
        "alts": []
      },
      {
        "keyword": "seychelles",
        "alts": []
      },
      {
        "keyword": "sierra leone",
        "alts": []
      },
      {
        "keyword": "singapore",
        "alts": []
      },
      {
        "keyword": "slovakia",
        "alts": []
      },
      {
        "keyword": "slovenia",
        "alts": []
      },
      {
        "keyword": "solomon islands",
        "alts": []
      },
      {
        "keyword": "somalia",
        "alts": []
      },
      {
        "keyword": "south africa",
        "alts": []
      },
      {
        "keyword": "south korea",
        "alts": ["republic of korea"]
      },
      {
        "keyword": "spain",
        "alts": []
      },
      {
        "keyword": "sudan",
        "alts": []
      },
      {
        "keyword": "suriname",
        "alts": []
      },
      {
        "keyword": "swaziland",
        "alts": ["eswatini"]
      },
      {
        "keyword": "sweden",
        "alts": []
      },
      {
        "keyword": "switzerland",
        "alts": []
      },
      {
        "keyword": "syria",
        "alts": []
      },
      {
        "keyword": "taiwan",
        "alts": []
      },
      {
        "keyword": "tajikistan",
        "alts": []
      },
      {
        "keyword": "tanzania",
        "alts": []
      },
      {
        "keyword": "thailand",
        "alts": []
      },
      {
        "keyword": "togo",
        "alts": []
      },
      {
        "keyword": "tonga",
        "alts": []
      },
      {
        "keyword": "trinidad and tobago",
        "alts": []
      },
      {
        "keyword": "tunisia",
        "alts": []
      },
      {
        "keyword": "turkey",
        "alts": []
      },
      {
        "keyword": "turkmenistan",
        "alts": []
      },
      {
        "keyword": "tuvalu",
        "alts": []
      },
      {
        "keyword": "uganda",
        "alts": []
      },
      {
        "keyword": "ukraine",
        "alts": []
      },
      {
        "keyword": "united arab emirates",
        "alts": []
      },
      {
        "keyword": "united kingdom",
        "alts": ["uk", "great britain"]
      },
      {
        "keyword": "united states of america",
        "alts": ["usa", "united states", "us"]
      },
      {
        "keyword": "uruguay",
        "alts": []
      },
      {
        "keyword": "uzbekistan",
        "alts": []
      },
      {
        "keyword": "vanuatu",
        "alts": []
      },
      {
        "keyword": "venezuela",
        "alts": []
      },
      {
        "keyword": "vietnam",
        "alts": []
      },
      {
        "keyword": "yemen",
        "alts": []
      },
      {
        "keyword": "zambia",
        "alts": []
      },
      {
        "keyword": "zimbabwe",
        "alts": []
      }
    ]
  },
  {
    "category": "city",
    "words": [
      {
        "keyword": "amsterdam netherlands",
        "alts": ["amsterdam", "amsterdam holland"]
      },
      {
        "keyword": "anaheim california",
        "alts": ["anaheim", "anaheim ca", "anaheim usa"]
      },
      {
        "keyword": "austin texas",
        "alts": ["austin", "austin tx", "austin usa"]
      },
      {
        "keyword": "auckland new zealand",
        "alts": ["auckland", "auckland nz"]
      },
      {
        "keyword": "asheville north carolina",
        "alts": ["asheville", "asheville nc", "asheville usa"]
      },
      {
        "keyword": "ashgabat turkmenistan",
        "alts": ["ashgabat", "ashgabad turkmenistan"]
      },
      {
        "keyword": "athens greece",
        "alts": ["athens"]
      },
      {
        "keyword": "athens georgia",
        "alts": ["athens", "athens ga", "athens usa"]
      },
      {
        "keyword": "atlanta georgia",
        "alts": ["atlanta", "atlanta ga", "atlanta usa"]
      },
      {
        "keyword": "antwerp belgium",
        "alts": ["antwerp"]
      },
      {
        "keyword": "adelaide australia",
        "alts": ["adelaide"]
      },
      {
        "keyword": "astana kazakhstan",
        "alts": ["astana"]
      },
      {
        "keyword": "asuncion paraguay",
        "alts": ["asuncion"]
      },
      {
        "keyword": "algiers algeria",
        "alts": ["algiers"]
      },
      {
        "keyword": "acapulco mexico",
        "alts": ["acapulco"]
      },
      {
        "keyword": "ankara turkey",
        "alts": ["ankara"]
      },
      {
        "keyword": "baghdad iraq",
        "alts": ["baghdad"]
      },
      {
        "keyword": "bangkok thailand",
        "alts": ["bangkok"]
      },
      {
        "keyword": "beijing china",
        "alts": ["beijing", "peking"]
      },
      {
        "keyword": "berlin germany",
        "alts": ["berlin"]
      },
      {
        "keyword": "boston massachusetts",
        "alts": ["boston", "boston ma", "boston usa"]
      },
      {
        "keyword": "buenos aires argentina",
        "alts": ["buenos aires"]
      },
      {
        "keyword": "bursa turkey",
        "alts": ["bursa"]
      },
      {
        "keyword": "bucharest romania",
        "alts": ["bucharest"]
      },
      {
        "keyword": "baltimore maryland",
        "alts": ["baltimore", "baltimore md", "baltimore usa"]
      },
      {
        "keyword": "beirut lebanon",
        "alts": ["beirut"]
      },
      {
        "keyword": "belfast northern ireland",
        "alts": ["belfast", "belfast uk", "belfast united kingdom"]
      },
      {
        "keyword": "bratislava slovakia",
        "alts": ["bratislava"]
      },
      {
        "keyword": "belgrade serbia",
        "alts": ["belgrade"]
      },
      {
        "keyword": "budapest hungary",
        "alts": ["budapest"]
      },
      {
        "keyword": "baku azerbaijan",
        "alts": ["baku"]
      },
      {
        "keyword": "bordeaux france",
        "alts": ["bordeaux"]
      },
      {
        "keyword": "busan south korea",
        "alts": ["busan"]
      },
      {
        "keyword": "brussels belgium",
        "alts": ["brussels"]
      },
      {
        "keyword": "bangalore india",
        "alts": ["bangalore"]
      },
      {
        "keyword": "calgary canada",
        "alts": ["calgary"]
      },
      {
        "keyword": "chicago illinois",
        "alts": ["chicago", "chi"]
      },
      {
        "keyword": "copenhagen denmark",
        "alts": ["copenhagen"]
      },
      {
        "keyword": "columbus ohio",
        "alts": ["columbus"]
      },
      {
        "keyword": "cologne germany",
        "alts": ["cologne"]
      },
      {
        "keyword": "cairo egypt",
        "alts": ["cairo"]
      },
      {
        "keyword": "cape town south africa",
        "alts": ["cape town", "cape town sa"]
      },
      {
        "keyword": "caracas venezuela",
        "alts": ["caracas"]
      },
      {
        "keyword": "cleveland ohio",
        "alts": ["cleveland"]
      },
      {
        "keyword": "cork ireland",
        "alts": ["cork"]
      },
      {
        "keyword": "christchurch new zealand",
        "alts": ["christchurch", "christchurch nz"]
      },
      {
        "keyword": "casablanca morocco",
        "alts": ["casablanca"]
      },
      {
        "keyword": "chengdu china",
        "alts": ["chengdu"]
      },
      {
        "keyword": "cannes france",
        "alts": ["cannes"]
      },
      {
        "keyword": "canberra australia",
        "alts": ["canberra"]
      },
      {
        "keyword": "dallas texas",
        "alts": ["dallas", "dallas tx"]
      },
      {
        "keyword": "dubai united arab emirates",
        "alts": ["dubai", "dubai uae"]
      },
      {
        "keyword": "dhaka bangladesh",
        "alts": ["dhaka"]
      },
      {
        "keyword": "dakar senegal",
        "alts": ["dakar"]
      },
      {
        "keyword": "delhi india",
        "alts": ["delhi"]
      },
      {
        "keyword": "durban south africa",
        "alts": ["durban"]
      },
      {
        "keyword": "dublin ireland",
        "alts": ["dublin"]
      },
      {
        "keyword": "dalian china",
        "alts": ["dalian"]
      },
      {
        "keyword": "doha qatar",
        "alts": ["doha"]
      },
      {
        "keyword": "denver colorado",
        "alts": ["denver", "denver co", "denver usa"]
      },
      {
        "keyword": "dusseldorf germany",
        "alts": ["dusseldorf"]
      },
      {
        "keyword": "davao city philippines",
        "alts": ["davao city"]
      },
      {
        "keyword": "darwin australia",
        "alts": ["darwin"]
      },
      {
        "keyword": "dunfermline scotland",
        "alts": ["dunfermline"]
      },
      {
        "keyword": "daegu south korea",
        "alts": ["daegu"]
      },
      {
        "keyword": "damascus syria",
        "alts": ["damascus"]
      },
      {
        "keyword": "dar es salaam tanzania",
        "alts": ["dar es salaam"]
      },
      {
        "keyword": "edinburgh scotland",
        "alts": ["edinburgh"]
      },
      {
        "keyword": "edmonton canada",
        "alts": ["edmonton"]
      },
      {
        "keyword": "essen germany",
        "alts": ["essen"]
      },
      {
        "keyword": "evora portugal",
        "alts": ["evora"]
      },
      {
        "keyword": "ensenada mexico",
        "alts": ["ensenada"]
      },
      {
        "keyword": "el paso texas",
        "alts": ["el paso"]
      },
      {
        "keyword": "enugu nigeria",
        "alts": ["enugu"]
      },
      {
        "keyword": "enschede netherlands",
        "alts": ["enschede"]
      },
      {
        "keyword": "eureka california",
        "alts": ["eureka"]
      },
      {
        "keyword": "erie pennsylvania",
        "alts": ["erie", "erie pa", "erie usa"]
      },
      {
        "keyword": "eilat israel",
        "alts": ["eilat"]
      },
      {
        "keyword": "essentuki russia",
        "alts": ["essentuki"]
      },
      {
        "keyword": "esbjerg denmark",
        "alts": ["esbjerg"]
      },
      {
        "keyword": "fez morocco",
        "alts": ["fez"]
      },
      {
        "keyword": "florence italy",
        "alts": ["florence"]
      },
      {
        "keyword": "frankfurt germany",
        "alts": ["frankfurt"]
      },
      {
        "keyword": "fort worth texas",
        "alts": ["fort worth", "forth worth tx", "fort worth usa"]
      },
      {
        "keyword": "fukuoka japan",
        "alts": ["fukuoka"]
      },
      {
        "keyword": "faisalabad pakistan",
        "alts": ["faisalabad"]
      },
      {
        "keyword": "fujairah united arab emirates",
        "alts": ["fujairah", "fujairah uae"]
      },
      {
        "keyword": "funafuti tuvalu",
        "alts": ["funafuti"]
      },
      {
        "keyword": "florianopolis brazil",
        "alts": ["florianopolis"]
      },
      {
        "keyword": "flinders australia",
        "alts": ["flinders"]
      },
      {
        "keyword": "faro portugal",
        "alts": ["faro"]
      },
      {
        "keyword": "fujairah united arab emirates",
        "alts": ["fujairah", "fujairah uae"]
      },
      {
        "keyword": "fort mcmurray canada",
        "alts": ["fort mcmurray"]
      },
      {
        "keyword": "fortaleza brazil",
        "alts": ["fortaleza"]
      },
      {
        "keyword": "friesland netherlands",
        "alts": ["friesland"]
      },
      {
        "keyword": "funchal portugal",
        "alts": ["funchal"]
      },
      {
        "keyword": "fuzhou china",
        "alts": ["fuzhou"]
      },
      {
        "keyword": "fresno california",
        "alts": ["fresno", "fresno ca", "fresno usa"]
      },
      {
        "keyword": "fermoy ireland",
        "alts": ["fermoy"]
      },
      {
        "keyword": "fukushima japan",
        "alts": ["fukushima"]
      },
      {
        "keyword": "glasgow scotland",
        "alts": ["glasgow"]
      },
      {
        "keyword": "guangzhou china",
        "alts": ["guangzhou"]
      },
      {
        "keyword": "gdansk poland",
        "alts": ["gdansk"]
      },
      {
        "keyword": "guatemala city guatemala",
        "alts": ["guatemala city"]
      },
      {
        "keyword": "guwahati india",
        "alts": ["guwahati"]
      },
      {
        "keyword": "gyeongju south korea",
        "alts": ["gyeongju"]
      },
      {
        "keyword": "genoa italy",
        "alts": ["genoa"]
      },
      {
        "keyword": "grahamstown south africa",
        "alts": ["grahamstown"]
      },
      {
        "keyword": "guadalajara mexico",
        "alts": ["guadalajara"]
      },
      {
        "keyword": "geneva switzerland",
        "alts": ["geneva"]
      },
      {
        "keyword": "graz austria",
        "alts": ["graz"]
      },
      {
        "keyword": "gwangju south korea",
        "alts": ["gwangju"]
      },
      {
        "keyword": "houston texas",
        "alts": ["houston", "houston tx", "houston usa"]
      },
      {
        "keyword": "hamburg germany",
        "alts": ["hamburg"]
      },
      {
        "keyword": "hanoi vietnam",
        "alts": ["hanoi"]
      },
      {
        "keyword": "helsinki finland",
        "alts": ["helsinki"]
      },
      {
        "keyword": "ho chi minh city vietnam",
        "alts": ["ho chi minh city", "ho chi minh", "saigon"]
      },
      {
        "keyword": "haifa israel",
        "alts": ["haifa"]
      },
      {
        "keyword": "havana cuba",
        "alts": ["havana"]
      },
      {
        "keyword": "hong kong china",
        "alts": ["hong kong"]
      },
      {
        "keyword": "hobart australia",
        "alts": ["hobart"]
      },
      {
        "keyword": "hangzhou china",
        "alts": ["hangzhou"]
      },
      {
        "keyword": "hilo hawaii",
        "alts": ["hilo"]
      },
      {
        "keyword": "hermosillo mexico",
        "alts": ["hermosillo"]
      },
      {
        "keyword": "honolulu hawaii",
        "alts": ["honolulu", "honolulu hi", "honolulu usa"]
      },
      {
        "keyword": "helsingborg sweden",
        "alts": ["helsingborg"]
      },
      {
        "keyword": "hiroshima japan",
        "alts": ["hiroshima"]
      },
      {
        "keyword": "harare zimbabwe",
        "alts": ["harare"]
      },
      {
        "keyword": "istanbul turkey",
        "alts": ["istanbul"]
      },
      {
        "keyword": "indianapolis indiana",
        "alts": ["indianapolis", "indianapolis in", "indianapolis usa"]
      },
      {
        "keyword": "ibadan nigeria",
        "alts": ["ibadan"]
      },
      {
        "keyword": "istanbul turkey",
        "alts": ["istanbul"]
      },
      {
        "keyword": "indore india",
        "alts": ["indore"]
      },
      {
        "keyword": "izmir turkey",
        "alts": ["izmir"]
      },
      {
        "keyword": "isafahan iran",
        "alts": ["isafahan"]
      },
      {
        "keyword": "incheon south korea",
        "alts": ["incheon"]
      },
      {
        "keyword": "innsbruck austria",
        "alts": ["innsbruck"]
      },
      {
        "keyword": "islamabad pakistan",
        "alts": ["islamabad"]
      },
      {
        "keyword": "ingolstadt germany",
        "alts": ["ingolstadt"]
      },
      {
        "keyword": "irvine california",
        "alts": ["irvine", "irvine ca", "irvine usa"]
      },
      {
        "keyword": "irkutsk russia",
        "alts": ["irkutsk"]
      },
      {
        "keyword": "jakarta indonesia",
        "alts": ["jakarta"]
      },
      {
        "keyword": "jerusalem israel",
        "alts": ["jerusalem"]
      },
      {
        "keyword": "jacksonville florida",
        "alts": ["jacksonville"]
      },
      {
        "keyword": "johannesburg south africa",
        "alts": ["johannesburg"]
      },
      {
        "keyword": "jabalpur india",
        "alts": ["jabalpur"]
      },
      {
        "keyword": "jinan china",
        "alts": ["jinan"]
      },
      {
        "keyword": "jeddah saudi arabia",
        "alts": ["jeddah"]
      },
      {
        "keyword": "jalapa guatemala",
        "alts": ["jalapa"]
      },
      {
        "keyword": "jackson mississippi",
        "alts": ["jackson"]
      },
      {
        "keyword": "juarez mexico",
        "alts": ["juarez"]
      },
      {
        "keyword": "jabalpur india",
        "alts": ["jabalpur"]
      },
      {
        "keyword": "jining china",
        "alts": ["jining"]
      },
      {
        "keyword": "kampala uganda",
        "alts": ["kampala"]
      },
      {
        "keyword": "kathmandu nepal",
        "alts": ["kathmandu"]
      },
      {
        "keyword": "kaunas lithuania",
        "alts": ["kaunas"]
      },
      {
        "keyword": "kuala lumpur malaysia",
        "alts": ["kuala lumpur"]
      },
      {
        "keyword": "kyoto japan",
        "alts": ["kyoto"]
      },
      {
        "keyword": "kagoshima japan",
        "alts": ["kagoshima"]
      },
      {
        "keyword": "karachi pakistan",
        "alts": ["karachi"]
      },
      {
        "keyword": "kiev ukraine",
        "alts": ["kiev", "kyiv", "kyiv ukraine"]
      },
      {
        "keyword": "kingston jamaica",
        "alts": ["kingston"]
      },
      {
        "keyword": "kolkata india",
        "alts": ["kolkata", "calcutta"]
      },
      {
        "keyword": "kunming china",
        "alts": ["kunming"]
      },
      {
        "keyword": "kabul afghanistan",
        "alts": ["kabul"]
      },
      {
        "keyword": "kyiv ukraine",
        "alts": ["kyiv"]
      },
      {
        "keyword": "kawasaki japan",
        "alts": ["kawasaki"]
      },
    
      {
        "keyword": "london england",
        "alts": ["london"]
      },
      {
        "keyword": "la paz bolivia",
        "alts": ["la paz"]
      },
      {
        "keyword": "los angeles california",
        "alts": ["los angeles", "los angeles ca", "los angeles usa",  "la" , "la ca", "la california", "la usa"]
      },
      {
        "keyword": "lima peru",
        "alts": ["lima"]
      },
      {
        "keyword": "lyon france",
        "alts": ["lyon"]
      },
      {
        "keyword": "lisbon portugal",
        "alts": ["lisbon"]
      },
      {
        "keyword": "luanda angola",
        "alts": ["luanda"]
      },
      {
        "keyword": "liverpool england",
        "alts": ["liverpool", "liverpool uk"]
      },
      {
        "keyword": "lagos nigeria",
        "alts": ["lagos"]
      },
      {
        "keyword": "leeds england",
        "alts": ["leeds", "leeds uk"]
      },
      {
        "keyword": "ljubljana slovenia",
        "alts": ["ljubljana"]
      },
      {
        "keyword": "lyon france",
        "alts": ["lyon"]
      },
      {
        "keyword": "lima peru",
        "alts": ["lima"]
      },
      {
        "keyword": "lviv ukraine",
        "alts": ["lviv"]
      },
      {
        "keyword": "leipzig germany",
        "alts": ["leipzig"]
      },
      {
        "keyword": "lusaka zambia",
        "alts": ["lusaka"]
      },
      {
        "keyword": "lausanne switzerland",
        "alts": ["lausanne"]
      },
      {
        "keyword": "madrid spain",
        "alts": ["madrid"]
      },
      {
        "keyword": "manchester england",
        "alts": ["manchester"]
      },
      {
        "keyword": "mexico city mexico",
        "alts": ["mexico city"]
      },
      {
        "keyword": "manila philippines",
        "alts": ["manila"]
      },
      {
        "keyword": "montreal canada",
        "alts": ["montreal"]
      },
      {
        "keyword": "milan italy",
        "alts": ["milan"]
      },
      {
        "keyword": "moscow russia",
        "alts": ["moscow"]
      },
      {
        "keyword": "madrid spain",
        "alts": ["madrid"]
      },
      {
        "keyword": "mumbai india",
        "alts": ["mumbai"]
      },
      {
        "keyword": "managua nicaragua",
        "alts": ["managua"]
      },
      {
        "keyword": "melbourne australia",
        "alts": ["melbourne"]
      },
      {
        "keyword": "marrakech morocco",
        "alts": ["marrakech"]
      },
      {
        "keyword": "miami florida",
        "alts": ["miami", "miami fl", "miami usa"]
      },
      {
        "keyword": "minneapolis minnesota",
        "alts": ["minneapolis"]
      },
      {
        "keyword": "mecca saudi arabia",
        "alts": ["mecca"]
      },
      {
        "keyword": "melbourne australia",
        "alts": ["melbourne"]
      },
      {
        "keyword": "makati philippines",
        "alts": ["makati"]
      },
      {
        "keyword": "monterrey mexico",
        "alts": ["monterrey"]
      },
      {
        "keyword": "nagoya japan",
        "alts": ["nagoya"]
      },
      {
        "keyword": "new york city",
        "alts": ["new york city", "nyc", "new york"]
      },
      {
        "keyword": "nanjing china",
        "alts": ["nanjing"]
      },
      {
        "keyword": "new delhi india",
        "alts": ["new delhi"]
      },
      {
        "keyword": "nantes france",
        "alts": ["nantes"]
      },
      {
        "keyword": "noida india",
        "alts": ["noida"]
      },
      {
        "keyword": "newcastle upon tyne england",
        "alts": ["newcastle", "newcastle upon tyne"]
      },
      {
        "keyword": "nice france",
        "alts": ["nice"]
      },
      {
        "keyword": "nurumberg germany",
        "alts": ["nurumberg"]
      },
      {
        "keyword": "new orleans louisiana",
        "alts": ["new orleans"]
      },
      {
        "keyword": "nairobi kenya",
        "alts": ["nairobi"]
      },
      {
        "keyword": "naples italy",
        "alts": ["naples"]
      },
      {
        "keyword": "noosa australia",
        "alts": ["noosa"]
      },
      {
        "keyword": "osaka japan",
        "alts": ["osaka"]
      },
      {
        "keyword": "oklahoma city oklahoma",
        "alts": ["oklahoma city"]
      },
      {
        "keyword": "oslo norway",
        "alts": ["oslo"]
      },
      {
        "keyword": "oxford england",
        "alts": ["oxford"]
      },
      {
        "keyword": "ottawa canada",
        "alts": ["ottawa"]
      },
      {
        "keyword": "orsay france",
        "alts": ["orsay"]
      },
      {
        "keyword": "odessa ukraine",
        "alts": ["odessa"]
      },
      {
        "keyword": "oranjestad aruba",
        "alts": ["oranjestad"]
      },
      {
        "keyword": "orlando florida",
        "alts": ["orlando", "orlando fl", "orlando usa"]
      },
      {
        "keyword": "ostrava czech republic",
        "alts": ["ostrava"]
      },
      {
        "keyword": "oaxaca mexico",
        "alts": ["oaxaca"]
      },
      {
        "keyword": "otago new zealand",
        "alts": ["otago"]
      },
      {
        "keyword": "ouagadougou burkina faso",
        "alts": ["ouagadougou"]
      },
      {
        "keyword": "odense denmark",
        "alts": ["odense"]
      },
      {
        "keyword": "oulu finland",
        "alts": ["oulu"]
      },
      {
        "keyword": "paris france",
        "alts": ["paris"]
      },
      {
        "keyword": "prague czech republic",
        "alts": ["prague"]
      },
      {
        "keyword": "porto portugal",
        "alts": ["porto"]
      },
      {
        "keyword": "philadelphia pennsylvania",
        "alts": ["philadelphia"]
      },
      {
        "keyword": "pyeongyang north korea",
        "alts": ["pyeongyang"]
      },
      {
        "keyword": "perth australia",
        "alts": ["perth"]
      },
      {
        "keyword": "plovdiv bulgaria",
        "alts": ["plovdiv"]
      },
      {
        "keyword": "pattaya thailand",
        "alts": ["pattaya"]
      },
      {
        "keyword": "portland oregon",
        "alts": ["portland"]
      },
      {
        "keyword": "phoenix arizona",
        "alts": ["phoenix"]
      },
      {
        "keyword": "porto alegre brazil",
        "alts": ["porto alegre"]
      },
      {
        "keyword": "peshawar pakistan",
        "alts": ["peshawar"]
      },
      {
        "keyword": "panama city panama",
        "alts": ["panama city"]
      },
      {
        "keyword": "rome italy",
        "alts": ["rome"]
      },
      {
        "keyword": "rio de janeiro brazil",
        "alts": ["rio de janeiro"]
      },
      {
        "keyword": "riyadh saudi arabia",
        "alts": ["riyadh"]
      },
      {
        "keyword": "reykjavik iceland",
        "alts": ["reykjavik"]
      },
      {
        "keyword": "rotterdam netherlands",
        "alts": ["rotterdam"]
      },
      {
        "keyword": "ras al khaimah united arab emirates",
        "alts": ["ras al khaimah"]
      },
      {
        "keyword": "raleigh north carolina",
        "alts": ["raleigh"]
      },
      {
        "keyword": "riga latvia",
        "alts": ["riga"]
      },
      {
        "keyword": "rochester new york",
        "alts": ["rochester"]
      },
      {
        "keyword": "recife brazil",
        "alts": ["recife"]
      },
      {
        "keyword": "san francisco california",
        "alts": ["san francisco"]
      },
      {
        "keyword": "sydney australia",
        "alts": ["sydney"]
      },
      {
        "keyword": "singapore",
        "alts": ["singapore"]
      },
      {
        "keyword": "seoul south korea",
        "alts": ["seoul"]
      },
      {
        "keyword": "stockholm sweden",
        "alts": ["stockholm"]
      },
      {
        "keyword": "santiago chile",
        "alts": ["santiago"]
      },
      {
        "keyword": "san diego california",
        "alts": ["san diego"]
      },
      {
        "keyword": "shanghai china",
        "alts": ["shanghai"]
      },
      {
        "keyword": "sao paulo brazil",
        "alts": ["sao paulo"]
      },
      {
        "keyword": "stuttgart germany",
        "alts": ["stuttgart"]
      },
      {
        "keyword": "sevilla spain",
        "alts": ["sevilla"]
      },
      {
        "keyword": "saskatoon canada",
        "alts": ["saskatoon"]
      },
      {
        "keyword": "san salvador el salvador",
        "alts": ["san salvador"]
      },
      {
        "keyword": "sofia bulgaria",
        "alts": ["sofia"]
      },
      {
        "keyword": "seattle washington",
        "alts": ["seattle"]
      },
      {
        "keyword": "tokyo japan",
        "alts": ["tokyo"]
      },
      {
        "keyword": "torino italy",
        "alts": ["torino"]
      },
      {
        "keyword": "tunis tunisia",
        "alts": ["tunis"]
      },
      {
        "keyword": "tashkent uzbekistan",
        "alts": ["tashkent"]
      },
      {
        "keyword": "toronto canada",
        "alts": ["toronto"]
      },
      {
        "keyword": "tirana albania",
        "alts": ["tirana"]
      },
      {
        "keyword": "tijuana mexico",
        "alts": ["tijuana"]
      },
      {
        "keyword": "turin italy",
        "alts": ["turin"]
      },
      {
        "keyword": "tokyo japan",
        "alts": ["tokyo"]
      },
      {
        "keyword": "thessaloniki greece",
        "alts": ["thessaloniki"]
      },
      {
        "keyword": "taegu south korea",
        "alts": ["taegu"]
      },
      {
        "keyword": "taksim turkey",
        "alts": ["taksim"]
      },
      {
        "keyword": "taipei taiwan",
        "alts": ["taipei"]
      },
      {
        "keyword": "tripoli libya",
        "alts": ["tripoli"]
      },
      {
        "keyword": "tokyo japan",
        "alts": ["tokyo"]
      },
      {
        "keyword": "ulaanbaatar mongolia",
        "alts": ["ulaanbaatar"]
      },
      {
        "keyword": "ubud indonesia",
        "alts": ["ubud"]
      },
      {
        "keyword": "uppsala sweden",
        "alts": ["uppsala"]
      },
      {
        "keyword": "urumqi china",
        "alts": ["urumqi"]
      },
      {
        "keyword": "vaduz liechtenstein",
        "alts": ["vaduz"]
      },
      {
        "keyword": "vancouver canada",
        "alts": ["vancouver"]
      },
      {
        "keyword": "valencia spain",
        "alts": ["valencia"]
      },
      {
        "keyword": "vigo spain",
        "alts": ["vigo"]
      },
      {
        "keyword": "valparaiso chile",
        "alts": ["valparaiso"]
      },
      {
        "keyword": "vladivostok russia",
        "alts": ["vladivostok"]
      },
      {
        "keyword": "vienna austria",
        "alts": ["vienna"]
      },
      {
        "keyword": "vilnius lithuania",
        "alts": ["vilnius"]
      },
      {
        "keyword": "villarreal spain",
        "alts": ["villarreal"]
      },
      {
        "keyword": "washington dc",
        "alts": ["washington dc"]
      },
      {
        "keyword": "westminster england",
        "alts": ["westminster"]
      },
      {
        "keyword": "wilmington delaware",
        "alts": ["wilmington"]
      },
      {
        "keyword": "wroclaw poland",
        "alts": ["wroclaw"]
      },
      {
        "keyword": "warsaw poland",
        "alts": ["warsaw"]
      },
      {
        "keyword": "wellington new zealand",
        "alts": ["wellington"]
      },
      {
        "keyword": "winnipeg manitoba",
        "alts": ["winnipeg"]
      },
      {
        "keyword": "warsaw poland",
        "alts": ["warsaw"]
      },
      {
        "keyword": "wuhan china",
        "alts": ["wuhan"]
      },
      {
        "keyword": "yokohama japan",
        "alts": ["yokohama"]
      },
      {
        "keyword": "york england",
        "alts": ["york"]
      },
      {
        "keyword": "yaounde cameroon",
        "alts": ["yaounde"]
      },
      {
        "keyword": "yuma arizona",
        "alts": ["yuma"]
      },
      {
        "keyword": "ypres belgium",
        "alts": ["ypres"]
      },
      {
        "keyword": "yakutsk russia",
        "alts": ["yakutsk"]
      },
      {
        "keyword": "yerevan armenia",
        "alts": ["yerevan"]
      },
      {
        "keyword": "yanbu saudi arabia",
        "alts": ["yanbu"]
      },
      {
        "keyword": "yogyakarta indonesia",
        "alts": ["yogyakarta"]
      },
      {
        "keyword": "yekaterinburg russia",
        "alts": ["yekaterinburg"]
      },
      {
        "keyword": "zacatecas mexico",
        "alts": ["zacatecas"]
      },
      {
        "keyword": "zunyi china",
        "alts": ["zunyi"]
      },
      {
        "keyword": "zincantan mexico",
        "alts": ["zincantan"]
      },
      {
        "keyword": "zagreb croatia",
        "alts": ["zagreb"]
      },
      {
        "keyword": "zeeland netherlands",
        "alts": ["zeeland"]
      },
      {
        "keyword": "zhongshan china",
        "alts": ["zhongshan"]
      },
      {
        "keyword": "zanzibar tanzania",
        "alts": ["zanzibar"]
      },
      {
        "keyword": "zurich switzerland",
        "alts": ["zurich"]
      },
      {
        "keyword": "zaragoza spain",
        "alts": ["zaragoza"]
      }
    ]
  },
  {
    "category": "landmark",
    "words": [
      {
        "keyword": "denali",
        "alts": ["mount mckinley", "mt mckinley"]
      },
      {
        "keyword": "mount saint lias",
        "alts": ["mt st elias"]
      },
      {
        "keyword": "mount whitney",
        "alts": ["mt whitney"]
      },
      {
        "keyword": "mount rainier",
        "alts": ["mt rainier"]
      },
      {
        "keyword": "iztaccihuatl",
        "alts": []
      },
      {
        "keyword": "grand teton",
        "alts": []
      },
      {
        "keyword": "gannett peak",
        "alts": []
      },
      {
        "keyword": "mount adams",
        "alts": ["mt adams"]
      },
      {
        "keyword": "mount saint helens",
        "alts": ["mt st helens"]
      },
      {
        "keyword": "mount shasta",
        "alts": ["mt shasta"]
      },
      {
        "keyword": "mount saint helens",
        "alts": ["mt st helens"]
      },
      {
        "keyword": "pikes peak",
        "alts": []
      },
      {
        "keyword": "aconcagua",
        "alts": []
      },
      {
        "keyword": "fitz roy",
        "alts": []
      },
      {
        "keyword": "cotopaxi",
        "alts": []
      },
      {
        "keyword": "chimborazo",
        "alts": []
      },
      {
        "keyword": "mont blanc",
        "alts": []
      },
      {
        "keyword": "zugspitze",
        "alts": []
      },
      {
        "keyword": "mount elbrus",
        "alts": ["mt elbrus"]
      },
      {
        "keyword": "mount etna",
        "alts": ["mt etna"]
      },
      {
        "keyword": "everest",
        "alts": ["mt everest", "mount everest"]
      },
      {
        "keyword": "k2",
        "alts": []
      },
      {
        "keyword": "lhotse",
        "alts": []
      },
      {
        "keyword": "makalu",
        "alts": []
      },
      {
        "keyword": "cho oyu",
        "alts": []
      },
      {
        "keyword": "manaslu",
        "alts": []
      },
      {
        "keyword": "annapurna",
        "alts": []
      },
      {
        "keyword": "dhaulagiri",
        "alts": []
      },
      {
        "keyword": "nanga parbat",
        "alts": []
      },
      {
        "keyword": "kangchenjunga",
        "alts": []
      },
      {
        "keyword": "mount fuji",
        "alts": ["fuji", "mt fuji"]
      },
      {
        "keyword": "kilimanjaro",
        "alts": ["mount kilimanjaro", "mt kilimanjaro"]
      },
      {
        "keyword": "meru",
        "alts": ["mount meru", "mt meru"]
      },
      {
        "keyword": "aoraki",
        "alts": ["mount cook", "mt cook"]
      },
      {
        "keyword": "haleakala",
        "alts": []
      },
      {
        "keyword": "puncak jaya",
        "alts": []
      },
      {
        "keyword": "sumantri",
        "alts": []
      },
      {
        "keyword": "amazon",
        "alts": ["amazonas"]
      },
      {
        "keyword": "colorado river",
        "alts": []
      },
      {
        "keyword": "dnieper",
        "alts": []
      },
      {
        "keyword": "ganges",
        "alts": ["ganga", "ganges river"]
      },
      {
        "keyword": "illinois river",
        "alts": []
      },
      {
        "keyword": "mississippi river",
        "alts": []
      },
      {
        "keyword": "nile",
        "alts": []
      },
      {
        "keyword": "rhine",
        "alts": []
      },
      {
        "keyword": "yangtze river",
        "alts": ["changjiang", "yangtze"]
      },
      {
        "keyword": "yellow river",
        "alts": ["huang he"]
      },
      {
        "keyword": "zambezi river",
        "alts": ["zambezi"]
      },
      {
        "keyword": "yenisei river",
        "alts": ["yenisei"]
      }
    ]
  }
]
"""



In [ ]:
keywords_data = json.loads(KEYWORDS_JSON)

# Create a list to hold rows for the DataFrame
rows = []

# Flatten the JSON structure into rows
for item in keywords_data:
    category = item["category"]
    for word in item["words"]:
        rows.append({
            "keyword": word["keyword"],
            "category": category,
            "continent": "",  # You can add continent data if available
            "first_letter": word["keyword"][0].upper()  # First letter in uppercase
        })

# Create DataFrame
keywords = pd.DataFrame(rows)
keywords.head()


In [ ]:

import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle

def wrap_text(text, max_width, font_size):
    words = text.split()
    lines = []
    line = ""
    for word in words:
        test_line = f"{line} {word}".strip()
        if len(test_line) * font_size > max_width and line:
            lines.append(line)
            line = word
        else:
            line = test_line
    if line:
        lines.append(line)
    return lines

def renderer(context):
    # Unpack context
    act = context['act']
    agents = context['agents']
    environment = context['environment']
    frame = context['frame']
    height = context.get('height', 800)
    interactive = context['interactive']
    is_interactive = context['is_interactive']
    parent = context['parent']
    step = context['step']
    update = context['update']
    width = context.get('width', 1200)

    # Common Dimensions
    max_width = 1200
    max_height = 800
    canvas_size = min(height, width)
    unit = 8
    offset = canvas_size * 0.1 if canvas_size > 400 else unit / 2
    cell_size = (canvas_size - offset * 2) / 3

    # Create figure and axis
    fig, ax = plt.subplots(figsize=(width / 100, height / 100))
    ax.set_xlim(0, width)
    ax.set_ylim(0, height)
    ax.invert_yaxis()
    ax.set_axis_off()

    # Get state for the current step
    try:
        steps = environment['steps']
        state = steps[step]
    except (IndexError, KeyError) as e:
        print(f"Error accessing environment steps: {e}")
        return

    print("State:", state)

    if not isinstance(state, dict):
        print("State is not properly defined.")
        return

    # Extract information from state
    try:
        observation = state['observation']
        team1_index = len(observation['questions']) - 1
        team2_index = len(observation['questions']) - 1

        team1_question = observation['questions'][team1_index] if team1_index >= 0 else ""
        team2_question = observation['questions'][team2_index] if team2_index >= 0 else ""

        team1_answer = observation['answers'][team1_index] if len(observation['questions']) == len(observation['answers']) and team1_index >= 0 else ""
        team2_answer = observation['answers'][team2_index] if len(observation['questions']) == len(observation['answers']) and team2_index >= 0 else ""

        team1_guess = observation['guesses'][team1_index] if len(observation['questions']) == len(observation['guesses']) and team1_index >= 0 else ""
        team2_guess = observation['guesses'][team2_index] if len(observation['questions']) == len(observation['guesses']) and team2_index >= 0 else ""

        team1_reward = str(state['reward']) if state['reward'] != 0 else ""
        team2_reward = str(state['reward']) if state['reward'] != 0 else ""
    except (IndexError, KeyError) as e:
        print(f"Error extracting data from state: {e}")
        return

    info = environment.get('info', {})
    team1_text = info.get('TeamNames', ['Team 1'])[0]
    team2_text = info.get('TeamNames', ['Team 2'])[1]

    # Canvas parameters
    padding = 20
    row_width = (min(max_width, width) - padding * 3 - 100) / 2
    label_x = padding
    team1_x = padding + 100
    team2_x = padding * 2 + row_width + 100
    line_height = 40
    label_y = 120
    question_y = 160
    answer_y = 200
    guess_y = 240
    score_y = 280

    # Font size
    font_size = 20

    # Draw text
    try:
        # Keyword Row
        ax.text(label_x, line_height, f"Keyword: {observation['keyword']}", fontsize=font_size, color='#FFFFFF')

        # Team Row
        ax.text(team1_x, line_height, team1_text, fontsize=font_size, color='#FFFFFF')
        ax.text(team2_x, line_height, team2_text, fontsize=font_size, color='#FFFFFF')

        # Question Row
        ax.text(label_x, question_y, "Question:", fontsize=font_size, color='#FFFFFF')
        wrapped_text1 = wrap_text(team1_question, row_width, font_size)
        wrapped_text2 = wrap_text(team2_question, row_width, font_size)
        for i, line in enumerate(wrapped_text1):
            ax.text(team1_x, question_y - i * line_height, line, fontsize=font_size, color='#FFFFFF')
        for i, line in enumerate(wrapped_text2):
            ax.text(team2_x, question_y - i * line_height, line, fontsize=font_size, color='#FFFFFF')

        # Answer Row
        ax.text(label_x, answer_y, "Answer:", fontsize=font_size, color='#FFFFFF')
        ax.text(team1_x, answer_y, team1_answer, fontsize=font_size, color='#FFFFFF')
        ax.text(team2_x, answer_y, team2_answer, fontsize=font_size, color='#FFFFFF')

        # Guess Row
        ax.text(label_x, guess_y, "Guess:", fontsize=font_size, color='#FFFFFF')
        ax.text(team1_x, guess_y, team1_guess, fontsize=font_size, color='#FFFFFF')
        ax.text(team2_x, guess_y, team2_guess, fontsize=font_size, color='#FFFFFF')

        # Reward Row
        ax.text(label_x, score_y, "Reward:", fontsize=font_size, color='#FFFFFF')
        ax.text(team1_x, score_y, team1_reward, fontsize=font_size, color='#FFFFFF')
        ax.text(team2_x, score_y, team2_reward, fontsize=font_size, color='#FFFFFF')

        plt.show()
    except Exception as e:
        print(f"Error during rendering: {e}")

# Example context for testing
context = {
    'act': lambda x: print(f"Action: {x}"),
    'agents': [],
    'environment': {
        'steps': [
            {'observation': {'questions': ['What is AI?', 'Define ML.'], 'answers': ['Artificial Intelligence', 'Machine Learning'], 'guesses': ['AI', 'ML'], 'keyword': 'AI'}, 'reward': 10},
        ],
        'info': {'TeamNames': ['Team Alpha', 'Team Beta']}
    },
    'frame': None,
    'height': 800,
    'interactive': True,
    'is_interactive': lambda: True,
    'parent': None,
    'step': 0,
    'update': lambda: None,
    'width': 1200
}

renderer(context)


In [ ]:
import pandas as pd
from transformers import AutoTokenizer, AutoModelForCausalLM, AutoConfig
import torch
import numpy as np
import gc
import os

# Global variables and configuration
model_name = "abacusai/Llama-3-Smaug-8B"  # Consider a smaller model if memory issues persist
VERBOSE = True  # Set to True for debugging output


# Load model and tokenizer with GPU support
def load_model_and_tokenizer(model_name):
    try:
        config = AutoConfig.from_pretrained(model_name)
        model = AutoModelForCausalLM.from_pretrained(
            model_name,
            config=config,
            trust_remote_code=True,
            device_map="auto"  # Load the model to the appropriate device
        )
        tokenizer = AutoTokenizer.from_pretrained(model_name)
        model.eval()  # Set model to evaluation mode
        return model, tokenizer
    except Exception as e:
        if VERBOSE:
            print(f"Error loading model: {e}")
        return None, None


# Function to interact with LLM for yes/no questions
def get_yes_no(question, keyword, tokenizer, model):
    try:
        # Construct the prompt message
        if keyword in keywords.keyword.values:
            row = keywords.loc[keywords.keyword == keyword].iloc[0]
            category = row.category
            continent = row.continent
            negate = {
                "city": "It is not a country. It is not a landmark.",
                "country": "It is not a city. It is not a landmark.",
                "landmark": "It is not a city. It is not a country.",
            }
            prompt = f"We are playing 20 questions. The keyword is {keyword}. It is a {category}. {negate[category]} This word has first letter {keyword[0]}. This {category} is located in {continent}. {question}"
        else:
            prompt = f"We are playing 20 questions. The keyword is {keyword}. It is a thing. It is not a city. It is not a country. It is not a landmark. This word has first letter {keyword[0]}. {question}"

        messages = [
            {"role": "system", "content": "Answer yes or no to the following question and nothing else."},
            {"role": "user", "content": prompt}
        ]

        # Apply chat template and tokenize
        text = tokenizer.apply_chat_template(
            messages,
            tokenize=False,
            add_generation_prompt=True
        )
        model_inputs = tokenizer([text], return_tensors="pt").to('cuda')

        pad_token_id = tokenizer.pad_token_id
        if pad_token_id is None:
            pad_token_id = tokenizer.eos_token_id

        # Generate response using mixed precision
        with torch.no_grad(), torch.cuda.amp.autocast():
            generated_ids = model.generate(
                model_inputs.input_ids,
                attention_mask=model_inputs.attention_mask,
                pad_token_id=pad_token_id,
                max_new_tokens=1
            )
        
        generated_ids = [
            output_ids[len(model_inputs.input_ids[0]):] for output_ids in generated_ids
        ]

        response = tokenizer.batch_decode(generated_ids, skip_special_tokens=True)[0]
        if not "yes" in response.lower():
            response = "no"
        else:
            response = "yes"

        return response
    
    except Exception as e:
        if VERBOSE:
            print(f"Error during agent execution: {e}")
        return "no"
    
def agent_fn(obs, cfg):
    global keywords

   

    # Initialize response variable
    response = "no"  # Set a default value for response
    
    try:
        # Ensure keywords are loaded correctly
       # try:
            #keywords = pd.read_csv("/kaggle/input/updated-kaggle-keywords/keywords_v2.csv")
        #except FileNotFoundError:
            #if VERBOSE:
               # print("Keywords file not found. Using an empty DataFrame.")
           # keywords = pd.DataFrame(columns=["keyword", "category", "continent", "first_letter"])  # Create an empty DataFrame with expected columns

        if keywords.empty:
            response = "Unable to process due to missing keywords."
            return response

        # Initialize necessary variables
        cat_guess = 0
        con_guess = 0
        let_guess = 0
        category_yes = []
        category_no = []
        continent_yes = []
        continent_no = []
        first_letter_yes = []
        first_letter_no = []
        extra_guess = None

        categories = ["city", "country", "landmark"]
        continents = ["Europe", "Asia", "North America", "Africa", "South America", "Australia"]

        if obs.turnType == "ask":
            if (cat_guess < 3) and (len(category_yes) == 0):
                response = f"Is the keyword the name of a {categories[cat_guess]}?"
                cat_guess += 1
            elif (con_guess < 6) and (len(continent_yes) == 0):
                category = "place"
                if len(category_yes) == 1: 
                    category = category_yes[0]
                response = f"Is the {category} located in {continents[con_guess]}?"
                con_guess += 1
            else:
                IDX = keywords.category.isin(category_yes)
                IDX = IDX & (keywords.continent.isin(continent_yes))
                first_letters = list(keywords.loc[IDX, "first_letter"].value_counts().index.values)
                if let_guess < len(first_letters):
                    response = f"Does the keyword begin with the letter {first_letters[let_guess]}?"
                else:
                    IDX = keywords.guess == 0
                    if len(category_yes) > 0: IDX = IDX & (keywords.category.isin(category_yes))
                    if len(category_no) > 0: IDX = IDX & (~keywords.category.isin(category_no))
                    if len(continent_yes) > 0: IDX = IDX & (keywords.continent.isin(continent_yes))
                    if len(continent_no) > 0: IDX = IDX & (~keywords.continent.isin(continent_no))
                    if len(first_letter_yes) > 0: IDX = IDX & (keywords.first_letter.isin(first_letter_yes))
                    if len(first_letter_no) > 0: IDX = IDX & (~keywords.first_letter.isin(first_letter_no))

                    try:
                        guess = keywords.loc[IDX].sample(1).index.values[0]
                        keywords.loc[guess, 'guess'] = 1
                        response = keywords.loc[guess, "keyword"]
                    except:
                        response = np.random.choice(keywords.keyword.values)
                    extra_guess = response
                    response = f"Is it {response}?"
                let_guess += 1

        elif obs.turnType == "guess":
            category_yes = []
            category_no = []
            for k in range(cat_guess):
                if obs.answers[k] == "yes":
                    category_yes.append(categories[k])
                else:
                    category_no.append(categories[k])
            if (cat_guess == 3) and (len(category_yes) == 0):
                category_yes = ["city", "country", "landmark"]
                category_no = []

            continent_yes = []
            continent_no = []
            for k in range(con_guess):
                if obs.answers[k + cat_guess] == "yes":
                    continent_yes.append(continents[k])
                else:
                    continent_no.append(continents[k])
            if (con_guess == 6) and (len(continent_yes) == 0):
                continent_yes = ["Europe", "Asia", "North America", "Africa", "South America", "Australia"]
                continent_no = []

            first_letter_yes = []
            first_letter_no = []
            for k in range(let_guess):
                if k >= len(first_letters): continue
                if obs.answers[k + cat_guess + con_guess] == "yes":
                    first_letter_yes.append(first_letters[k])    
                else:
                    first_letter_no.append(first_letters[k])

            IDX = keywords.guess == 0
            if len(category_yes) > 0: IDX = IDX & (keywords.category.isin(category_yes))
            if len(category_no) > 0: IDX = IDX & (~keywords.category.isin(category_no))
            if len(continent_yes) > 0: IDX = IDX & (keywords.continent.isin(continent_yes))
            if len(continent_no) > 0: IDX = IDX & (~keywords.continent.isin(continent_no))
            if len(first_letter_yes) > 0: IDX = IDX & (keywords.first_letter.isin(first_letter_yes))
            if len(first_letter_no) > 0: IDX = IDX & (~keywords.first_letter.isin(first_letter_no))

            try:
                guess = keywords.loc[IDX].sample(1).index.values[0]
                keywords.loc[guess, 'guess'] = 1
                response = keywords.loc[guess, "keyword"]
            except:
                response = np.random.choice(keywords.keyword.values)

            if (let_guess > 0) and (let_guess >= len(first_letters)) and (obs.answers[-1] == "yes"):
                response = extra_guess

        elif obs.turnType == "answer":
            if obs.keyword.lower() in obs.questions[-1].lower():
                response = "yes"
            else:
                response = get_yes_no(obs.questions[-1], obs.keyword, tokenizer, model)

        else:
            response = "yes"  # Default response if all else fails

    except Exception as e:
        if VERBOSE:
            print(f"Error during agent execution: {e}")

    return response  # Always return response, even if it's the default value

class Observation:
    def __init__(self, turnType, answers=None, keyword=None, questions=None):
        self.turnType = turnType
        self.answers = answers or []
        self.keyword = keyword
        self.questions = questions or []

# Example usage
if __name__ == "__main__":
    model, tokenizer = load_model_and_tokenizer(model_name)
    
    observation_object = Observation(
        turnType="ask",
        questions=["Is it a country?"]  # Example question
    )
    configuration_object = {}  # Add necessary configuration parameters
    
    response = agent_fn(observation_object, configuration_object)
    print(f"Agent response: {response}")

    # Clean up resources
    del model, tokenizer
    gc.collect()
    torch.cuda.empty_cache()


In [ ]:
!apt install pigz pv > /dev/null

In [ ]:
!tar --use-compress-program='pigz --fast --recursive | pv' -cf /kaggle/working/submission.tar.gz -C /kaggle/input/gemma-2/pytorch/gemma-2-2b-it/1 .